## Simple Slack Bot Setup

Getting a first slack bot up and running

#### First, Python packages

We'll need the slackclient library from the Slack team to access the API

pip install slackclient

#### Setup a Slack Team

https://api.slack.com/


#### Setup the Slackbot on Slack 

https://api.slack.com/bot-users

Copy the Slackbot API Token


### Get the Bot's ID 

The Bot's ID is used to determine if messages parsed from the Slack RTM (real time messaging) API are being directed at our slack bot.

In [3]:
import time
from slackclient import SlackClient

BOT_NAME = 'shopbot'

# your bot's token here:
BOT_TOKEN = 'Your bot token here'

slack_client = SlackClient(BOT_TOKEN)

if __name__ == "__main__":
    api_call = slack_client.api_call("users.list")
    if api_call.get('ok'):
        # retrieve all users so we can find our bot
        users = api_call.get('members')
        for user in users:
            if 'name' in user and user.get('name') == BOT_NAME:
                print('User:',user['name'],'is a bot?',user['is_bot'])
                print("Bot ID for " + user['name'] +' is '+user.get('id'))
                # set BOT_ID for team specific bot ID
                BOT_ID = user.get('id')
    else:
        print("could not find bot user with the name " + BOT_NAME)

User: shopbot is a bot? True
Bot ID for shopbot is U281D2V62


### Now Setup the Slack client

The Slack client will connect to the Slack RTM API and loop continuously checking for commands directed at the Slack Bot specified by the BOT_TOKEN

The AT_BOT variable is how we'll interact with the Slackbot  
In this case we'd have to enter '@shopbot: sup' in Slack to get a response.

In [ ]:
# constants
AT_BOT = "<@" + BOT_ID + ">:"
EXAMPLE_COMMAND = "sup"

We'll simplify the Bot reference command to @BOT_NAME and avoid having to delete the default white space and add a colon :

In [ ]:
AT_BOT = "<@" + BOT_ID + ">"

In [ ]:
slack_client = SlackClient(BOT_TOKEN)

def handle_command(command, channel):
    """
        Receives commands directed at the bot and determines if they
        are valid commands. If so, then acts on the commands. If not,
        returns back what it needs for clarification.
    """
    print("handle_command")
    response = "Nope.  I only do *" + EXAMPLE_COMMAND + \
               "* for now."
    if command.startswith(EXAMPLE_COMMAND):
        response = "What up!"
    slack_client.api_call("chat.postMessage", channel=channel,
                          text=response, as_user=True)


def parse_slack_output(slack_rtm_output):
    """
        The Slack Real Time Messaging API is an events firehose.
        this parsing function returns None unless a message is
        directed at the Bot, based on its ID.
    """
    output_list = slack_rtm_output
    if output_list and len(output_list) > 0:
        for output in output_list:
            if output and 'text' in output and AT_BOT in output['text']:
                # return text after the @ mention, whitespace removed
                return output['text'].split(AT_BOT)[1].strip().lower(), \
                       output['channel']
    return None, None


if __name__ == "__main__":
    READ_WEBSOCKET_DELAY = 1 # 1 second delay between reading from firehose
    if slack_client.rtm_connect():
        print(BOT_NAME,'connected and running!')
        while True:
            command, channel = parse_slack_output(slack_client.rtm_read())
            if command and channel:
                handle_command(command, channel)
            time.sleep(READ_WEBSOCKET_DELAY)
    else:
        print("Connection failed. Invalid Slack Token or Bot ID?")

 ### Now to add functionality 
 
 

#### Reference
Matt Makai (2016): https://www.fullstackpython.com/blog/build-first-slack-bot-python.html